In [1]:
import os

import argparse
import time


import torch
from torch.utils.data import DataLoader,Dataset
import torch.nn as nn
from torch.nn import BCELoss
import torch.nn.functional as F
from torch.nn.functional import binary_cross_entropy
import torch.nn.init as init
import torch.optim as optim
import argparse

import numpy as np

import matplotlib.pyplot as plt

from vnetpyt import vnet as Vnet

from mod3DUnet import Modified3DUNet as UNet


In [2]:
!nvidia-smi

Fri Aug 16 04:02:38 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.104      Driver Version: 410.104      CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-DGXS...  On   | 00000000:0E:00.0 Off |                    0 |
| N/A   36C    P0    49W / 300W |   4571MiB / 32478MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
from niftynet.engine.sampler_grid_v2 import GridSampler
from niftynet.engine.sampler_uniform_v2 import UniformSampler
from niftynet.engine.signal import TRAIN, VALID, INFER
from niftynet.engine.windows_aggregator_grid import GridSamplesAggregator
from niftynet.engine.windows_aggregator_base import ImageWindowsAggregator
from niftynet.engine.sampler_balanced_v2 import BalancedSampler
from niftynet.evaluation.pairwise_measures import PairwiseMeasures
from niftynet.io.image_reader import ImageReader
from niftynet.io.image_sets_partitioner import ImageSetsPartitioner
from niftynet.layer.mean_variance_normalisation import MeanVarNormalisationLayer
from niftynet.layer.pad import PadLayer


W0816 04:02:39.916503 140345729128256 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/niftynet/__init__.py:36: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

W0816 04:02:39.987415 140345729128256 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/niftynet/utilities/util_import.py:28: The name tf.logging.fatal is deprecated. Please use tf.compat.v1.logging.fatal instead.

E0816 04:02:39.989434 140345729128256 util_import.py:36] CRITICAL - Optional Python module cv2 not found, please install cv2 and retry if the application fails.
E0816 04:02:39.990676 140345729128256 util_import.py:36] CRITICAL - Optional Python module skimage.io not found, please install skimage.io and retry if the application fails.
W0816 04:02:40.059309 140345729128256 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/niftynet/io/misc_io.py:645: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphK

W0816 04:02:40.756999 140345729128256 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/niftynet/engine/application_variables.py:20: The name tf.summary.scalar is deprecated. Please use tf.compat.v1.summary.scalar instead.



W0816 04:02:40.758990 140345729128256 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/niftynet/engine/application_variables.py:21: The name tf.summary.histogram is deprecated. Please use tf.compat.v1.summary.histogram instead.



W0816 04:02:40.760592 140345729128256 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/niftynet/engine/application_variables.py:22: The name tf.summary.image is deprecated. Please use tf.compat.v1.summary.image instead.



In [4]:
class DatasetNiftySampler(Dataset):
    """
    A simple adapter
    converting NiftyNet sampler's output into PyTorch Dataset properties
    """
    def __init__(self, sampler):
        super(DatasetNiftySampler, self).__init__()
        self.sampler = sampler

    def __getitem__(self, index):
        data = self.sampler(idx=index)

        # Transpose to PyTorch format
        image = np.transpose(data['image'], (0, 5, 1, 2, 3, 4))
        label = np.transpose(data['label'], (0, 5, 1, 2, 3, 4))

        image = torch.from_numpy(image).float()
        label = torch.from_numpy(label).float()

        return image, label

    def __len__(self):
        return len(self.sampler.reader.output_list)

In [5]:
class SoftDiceLoss(nn.Module):

    def __init__(self):
        super(SoftDiceLoss, self).__init__()

    def forward(self, output, label):
        probs = output.view(-1)
        mask = label.view(-1)
        smooth = 1
        intersection = torch.sum(probs * mask)
        den1 = torch.sum(probs)
        den2 = torch.sum(mask)
        soft_dice = (2 * intersection + smooth) / (den1 + den2 + smooth)
        return -soft_dice


def dice(input, target):
    epsilon = 1e-8
    iflat = input.view(-1)
    tflat = target.view(-1)
    intersection = (iflat * tflat).sum()
    return 2 * intersection / (iflat.sum() + tflat.sum() + epsilon)

In [6]:
class tversky(nn.Module):
    
    def __init__(self):
        super(tversky,self).__init__()
        
    def forward(self,output,label):
        alpha = 0.1
        beta  = 0.9

        one = torch.ones_like(label)
        p0 = output      # proba that voxels are class i
        p1 = one-output # proba that voxels are not class i
        g0 = label
        g1 = one-label

        num = torch.sum(p0*g0, (0,1,2,3))
        den = num + alpha*torch.sum(p0*g1,(0,1,2,3)) + beta*torch.sum(p1*g0,(0,1,2,3))

        T = torch.sum(num/den) # when summing over classes, T has dynamic range [0 Ncl]

        Ncl = torch.reshape(label, (-1,)).to(dtype = torch.float32)
        #Ncl = K.cast(K.shape(y_true)[-1], 'float32')
        return Ncl-T

In [7]:
def compute_per_channel_dice(input, target, epsilon=1e-5, ignore_index=None, weight=None):
    # assumes that input is a normalized probability

    # input and target shapes must match
    assert input.size() == target.size(), "'input' and 'target' must have the same shape"

    # mask ignore_index if present
    if ignore_index is not None:
        mask = target.clone().ne_(ignore_index)
        mask.requires_grad = False

        input = input * mask
        target = target * mask

    input = flatten(input)
    target = flatten(target)

    target = target.float()
    # Compute per channel Dice Coefficient
    intersect = (input * target).sum(-1)
    if weight is not None:
        intersect = weight * intersect

    denominator = (input + target).sum(-1)
    return 2. * intersect / denominator.clamp(min=epsilon)

In [8]:
class GeneralizedDiceLoss(nn.Module):
    """Computes Generalized Dice Loss (GDL) as described in https://arxiv.org/pdf/1707.03237.pdf
    """

    def __init__(self, epsilon=1e-5, weight=None, ignore_index=None, sigmoid_normalization=True):
        super(GeneralizedDiceLoss, self).__init__()
        self.epsilon = epsilon
        self.register_buffer('weight', weight)
        self.ignore_index = ignore_index
        if sigmoid_normalization:
            self.normalization = nn.Sigmoid()
        else:
            self.normalization = nn.Softmax(dim=1)

    def forward(self, input, target):
        # get probabilities from logits
        input = self.normalization(input)

        assert input.size() == target.size(), "'input' and 'target' must have the same shape"

        # mask ignore_index if present
        if self.ignore_index is not None:
            mask = target.clone().ne_(self.ignore_index)
            mask.requires_grad = False

            input = input * mask
            target = target * mask

        input = torch.flatten(input)
        target = torch.flatten(target)

        target = target.float()
        target_sum = target.sum(-1)
        class_weights = torch.autograd.Variable(1. / (target_sum * target_sum).clamp(min=self.epsilon), requires_grad=False)

        intersect = (input * target).sum(-1) * class_weights
        if self.weight is not None:
            weight = torch.autograd.Variable(self.weight, requires_grad=False)
            intersect = weight * intersect
        intersect = intersect.sum()

        denominator = ((input + target).sum(-1) * class_weights).sum()

        return 1. - 2. * intersect / denominator.clamp(min=self.epsilon)

In [9]:
class DiceCoefficient:
    """Computes Dice Coefficient.
    Generalized to multiple channels by computing per-channel Dice Score
    (as described in https://arxiv.org/pdf/1707.03237.pdf) and theTn simply taking the average.
    Input is expected to be probabilities instead of logits.
    This metric is mostly useful when channels contain the same semantic class (e.g. affinities computed with different offsets).
    DO NOT USE this metric when training with DiceLoss, otherwise the results will be biased towards the loss.
    """

    def __init__(self, epsilon=1e-5, ignore_index=None, **kwargs):
        self.epsilon = epsilon
        self.ignore_index = ignore_index

    def __call__(self, input, target):
        """
        :param input: 5D probability maps torch tensor (NxCxDxHxW)
        :param target: 4D or 5D ground truth torch tensor. 4D (NxDxHxW) tensor will be expanded to 5D as one-hot
        :return: Soft Dice Coefficient averaged over all channels/classes
        """
        # Average across channels in order to get the final score
        return torch.mean(compute_per_channel_dice(input, target, epsilon=self.epsilon, ignore_index=self.ignore_index))

In [10]:
data_split_file_new = 'fcd_train_val_infer_split.csv'

patch_size = (32,32,32)

in_channels = 3 #3?

n_classes = 1

num_epochs = 50

lr = 1e-3

ratios =[0.1,0.1]

batch_size = 5

cp_path = './CP2.pth'

image_path = 'Fcd_Data/'

label_path = 'Fcd_Data/Labels'

pred_path = 'Fcd_Data/pred_brain'


In [11]:
def get_reader(data_param,task_param,image_sets_partitioner, phase):
    
    if phase == 'training':
        image_reader = ImageReader().initialise(
            data_param,task_param = grouping_param,file_list=image_sets_partitioner.get_file_list(TRAIN))


    elif phase == 'validation':
        image_reader = ImageReader().initialise(
            data_param,task_param = grouping_param, file_list=image_sets_partitioner.get_file_list(VALID))
        #_, image_data, _ = image_reader(idx=0)


    elif phase == 'inference':
        image_reader = ImageReader().initialise(
            data_param,task_param = grouping_param,file_list=image_sets_partitioner.get_file_list(INFER))
        #_, image_data, _ = image_reader(idx=0)

    else:
        raise Exception('Invalid phase choice: {}'.format(
            {'phase': ['train', 'validation', 'inference']}))

    
    mean_variance_norm_layer = MeanVarNormalisationLayer(image_name='image')
    pad_layer = PadLayer(image_name=('image', 'label'), border=(8, 8, 8))
    image_reader.add_preprocessing_layers([mean_variance_norm_layer])

    if phase == 'inference':
        image_reader.add_preprocessing_layers([pad_layer])

    return image_reader

In [12]:
def get_sampler(image_reader, patch_size, phase):
    if phase in ('training', 'validation'):
        sampler = UniformSampler(image_reader,
                                 window_sizes=patch_size,
                                 windows_per_image=2)
    elif phase == 'inference':
        sampler = GridSampler(image_reader,
                              window_sizes=patch_size,
                              window_border=(8, 8, 8),
                              batch_size=1)
    else:
        raise Exception('Invalid phase choice: {}'.format(
            {'phase': ['train', 'validation', 'inference']}))

    return sampler

In [13]:
def train(dsets, model, optimizer,
          num_epochs, device, cp_path, batch_size):
    since = time.time()

    dataloaders = {
        x: DataLoader(dsets[x], batch_size=batch_size,
                      shuffle=True, num_workers=4)
        for x in ['training', 'validation']}

    model = model.to(device)

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['training', 'validation']:
            if phase == 'training':
                model.train()  # Set model to training mode
            else:
                model.eval()  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0.0
            epoch_samples = 0
            
            # Iterate over data
            for iteration, (inputs, labels) in enumerate(dataloaders[phase], 1):

                nbatches, wsize, nchannels, x, y, z, _ = inputs.size()

                inputs = inputs.view(nbatches * wsize, nchannels, x, y, z)
                labels = labels.view(nbatches * wsize, 1, x, y, z)

                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'training'):
                    outputs = model(inputs)
                    pred = (outputs > 0.5)

                    loss = binary_cross_entropy(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'training':
                        loss.backward()
                        optimizer.step()

                # statistics
                epoch_samples += inputs.size(0)
                running_loss += loss.item() * inputs.size(0)
                measures = PairwiseMeasures(
                    pred.cpu().numpy(), labels.cpu().numpy())
              

            epoch_loss = running_loss / epoch_samples

            epoch_acc = running_corrects / epoch_samples

            print('{} Loss: {:.4f} Dice: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

                

            if epoch == 0:
                best_loss = epoch_loss
                torch.save(model.state_dict(), cp_path.format(epoch + 1))

            # deep copy the model
            if phase == 'validation' and epoch_loss < best_loss:
                best_loss = epoch_loss
                torch.save(model.state_dict(), cp_path)
                print('Checkpoint {} saved!'.format(epoch + 1))

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))

In [14]:
def inference(sampler, model, device, pred_path, cp_path):
    output = GridSamplesAggregator(image_reader=sampler.reader,
                                   window_border=(8, 8, 8),
                                   output_path=pred_path)
    for _ in sampler():  # for each subject

        model.load_state_dict(torch.load(cp_path))
        model.to(device)
        model.eval()

        for batch_output in sampler():  # for each sliding window step
            window = batch_output['image']
            # [...,0,:] eliminates time coordinate from NiftyNet Volume
            window = window[..., 0, :]
            window = np.transpose(window, (0, 4, 1, 2, 3))
            window = torch.Tensor(window).to(device)

            with torch.no_grad():
                outputs = model(window)

            outputs = outputs.cpu().numpy()
            outputs = np.transpose(outputs, (0, 2, 3, 4, 1))
            output.decode_batch({'window_image': outputs.astype(np.float32)},
                                batch_output['image_location'])

In [15]:
print("[INFO]Reading data")
    # Dictionary with data parameters for NiftyNet Reader
if torch.cuda.is_available():
    print('[INFO] GPU available')
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

else:
    raise Exception("[INFO] No GPU found or Wrong gpu id")

    # Dictionary with data parameters for NiftyNet Reader
data_param = {
        'modal': {'path_to_search': os.path.join(image_path, 'images'), 'filename_contains': 'reg'},
        'interhemi': {'path_to_search': os.path.join(image_path, 'Interhemi'), 'filename_contains': 'interasymm'},
        'pvms': {'path_to_search': os.path.join(image_path, 'Pvms'), 'filename_contains' : 'pve'},
        'labels': {'path_to_search': label_path, 'filename_contains': 'label'}}

grouping_param = {'image': ('modal', 'interhemi', 'pvms'), 
                  'label': ('labels',)}


image_sets_partitioner = ImageSetsPartitioner().initialise(
        data_param=data_param,
        data_split_file=data_split_file_new,
        new_partition=False,
        ratios = ratios)

readers = {x: get_reader(data_param,grouping_param,image_sets_partitioner, x) for x in ['training', 'validation', 'inference']}
samplers = {x: get_sampler(readers[x], patch_size, x) for x in ['training', 'validation', 'inference']}

    # Training stage only
dsets = {x: DatasetNiftySampler(sampler=samplers[x])
             for x in ['training', 'validation']}

print("[INFO] Building model")
model = UNet(in_channels,n_classes)
#criterion = binary_cross_entropy(output,labels)
optimizer = optim.Adam(model.parameters(), lr=lr)

print("[INFO] Training")
train(dsets, model,optimizer,num_epochs, device, cp_path, batch_size)

print("[INFO] Inference")
inference(samplers['inference'], model, device, pred_path, cp_path)

[INFO]Reading data
[INFO] GPU available



W0816 04:02:40.903449 140345729128256 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/niftynet/io/image_sets_partitioner.py:269: The name tf.logging.debug is deprecated. Please use tf.compat.v1.logging.debug instead.



W0816 04:02:40.927857 140345729128256 image_sets_partitioner.py:368] Loading from existing partitioning file fcd_train_val_infer_split.csv, ignoring partitioning ratios.


INFO:niftynet: 

Number of subjects 5, input section names: ['subject_id', 'modal', 'interhemi', 'pvms', 'labels']
Dataset partitioning:
-- training 3 cases (60.00%),
-- validation 1 cases (20.00%),
-- inference 1 cases (20.00%).



I0816 04:02:40.933712 140345729128256 image_sets_partitioner.py:90] 

Number of subjects 5, input section names: ['subject_id', 'modal', 'interhemi', 'pvms', 'labels']
Dataset partitioning:
-- training 3 cases (60.00%),
-- validation 1 cases (20.00%),
-- inference 1 cases (20.00%).



...

INFO:niftynet: yielding 3072 locations from image


I0816 04:15:04.961242 140345729128256 sampler_grid_v2.py:87] yielding 3072 locations from image


CRITICAL:niftynet: No feasible samples from [Layer] grid_sampler


E0816 04:15:04.971232 140345729128256 sampler_grid_v2.py:124] CRITICAL - No feasible samples from [Layer] grid_sampler


UnboundLocalError: local variable 'output_dict' referenced before assignment